In [1]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Shrikrishna-jadhavar/Data-Science-Material/main/Dataset/Churn_Modelling.csv')

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'], dtype = int)
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [8]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [9]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [10]:
x

array([[-0.32622142,  0.29351742, -1.04175968, ..., -0.57380915,
         1.09598752, -1.09598752],
       [-0.44003595,  0.19816383, -1.38753759, ...,  1.74273971,
         1.09598752, -1.09598752],
       [-1.53679418,  0.29351742,  1.03290776, ..., -0.57380915,
         1.09598752, -1.09598752],
       ...,
       [ 0.60498839, -0.27860412,  0.68712986, ..., -0.57380915,
         1.09598752, -1.09598752],
       [ 1.25683526,  0.29351742, -0.69598177, ..., -0.57380915,
        -0.91241915,  0.91241915],
       [ 1.46377078, -1.04143285, -0.35020386, ..., -0.57380915,
         1.09598752, -1.09598752]])

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [13]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))
ann.add(Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7103 - accuracy: 0.5789
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7092 - accuracy: 0.5801
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.7079 - accuracy: 0.5817
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7065 - accuracy: 0.5822
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7052 - accuracy: 0.5832
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7038 - accuracy: 0.5842
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7024 - accuracy: 0.5853
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7009 - accuracy: 0.5865
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6995 - accuracy: 0.5874
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6981 - accura

In [14]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 2ms/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [ True]])

In [15]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.83      0.76      0.79      1585
           1       0.31      0.43      0.36       415

    accuracy                           0.69      2000
   macro avg       0.57      0.59      0.58      2000
weighted avg       0.73      0.69      0.70      2000



In [16]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 0s 1ms/step - loss: 0.5931 - accuracy: 0.6826


[0.5931447148323059, 0.6826249957084656]

# Hyperparameter tuning


1. Create the model: Start with an empty model.
2. Add layers: Depending on the number of layers (between 1 and 10), add layers to the model.
  Each layer has:
                  1. A number of units (8 to 128, increasing by 8).
                  2.An activation function (can be 'relu', 'tanh', or 'sigmoid').
                  3.The first layer also needs to know the input size (13 features).
3. Output layer: Add a final layer with 1 unit and 'sigmoid' activation, which is common for binary classification.
4. Compile the model: Choose an optimizer ('adam', 'rmsprop', or 'sgd'), use 'binary_crossentropy' as the loss function, and track 'accuracy'.
5. Return the model: The function returns the complete model.

In [17]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [18]:
x.shape

(10000, 13)

In [19]:
def hyper(hp):
    model = Sequential()

    # Loop to add layers
    for i in range(hp.Int('num_layers', 1, 10)):
        units = hp.Int(f'units_{i}', 8, 128, 8)  # Number of units in the layer
        activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid'])  # Activation function

        if i == 0:
            # First layer needs input dimension
            model.add(Dense(units, activation=activation, input_dim=13))
        else:
            # Other layers
            model.add(Dense(units, activation=activation))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [20]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [21]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7929999828338623

Best val_accuracy So Far: 0.859499990940094
Total elapsed time: 00h 00m 18s


In [22]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units_0': 40,
 'activation_0': 'relu',
 'optimizer': 'adam',
 'units_1': 48,
 'activation_1': 'tanh',
 'units_2': 16,
 'activation_2': 'relu',
 'units_3': 32,
 'activation_3': 'relu',
 'units_4': 112,
 'activation_4': 'relu',
 'units_5': 72,
 'activation_5': 'tanh',
 'units_6': 104,
 'activation_6': 'relu',
 'units_7': 80,
 'activation_7': 'tanh',
 'units_8': 48,
 'activation_8': 'tanh'}

In [23]:
x.shape

(10000, 13)

In [24]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [25]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 2s 3ms/step - loss: 0.3425 - accuracy: 0.8620 - val_loss: 0.3451 - val_accuracy: 0.8595
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3366 - accuracy: 0.8636 - val_loss: 0.3450 - val_accuracy: 0.8610
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3341 - accuracy: 0.8629 - val_loss: 0.3432 - val_accuracy: 0.8610
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3297 - accuracy: 0.8651 - val_loss: 0.3390 - val_accuracy: 0.8610
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3288 - accuracy: 0.8666 - val_loss: 0.3420 - val_accuracy: 0.8605
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3259 - accuracy: 0.8652 - val_loss: 0.3444 - val_accuracy: 0.8595
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3254 - accuracy: 0.8633 - val_loss: 0.3428 - val_accuracy: 0.8655
Epoch 

In [26]:
tuned_model.evaluate(xtrain,ytrain)

250/250 [==============================] - 0s 2ms/step - loss: 0.2581 - accuracy: 0.8939


[0.2580595910549164, 0.893875002861023]

In [27]:
tuned_model.evaluate(xtest,ytest)

63/63 [==============================] - 0s 2ms/step - loss: 0.3897 - accuracy: 0.8540


[0.3897467255592346, 0.8539999723434448]